# Information
Predict if a customer leave the bank:  
Churn rate of a bank: the annual percentage rate at which customers stop subscribing to a service.  

The columns in order:   
Row Number  
Customer ID  
Lastname  
Credit Score  
Geography  
Gender  
Age  
Tenure  
Balance  
Number of Products  
Has Credit Card  
Is Active Member  
Estimated Salary  
Exited: They wait 6 months and enter the exit information for each customer.  

You can apply this model to any dataset with lots of variables and two outcome : 0 or 1

This is a classification problem


# Part 1: Data Preprocessing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 1) IMPORT THE DATASET
dataset = pd.read_csv('Churn_Modelling.csv')

X = dataset.iloc[:, 3:13].values # we excluded the row number, customer id, surname columns. they have no impact for a customer to decide to leave or stay. 
y = dataset.iloc[:, 13].values #the targets, actual outputs
#.values: runs only the values, no headings.

# Classification template

#According to template, after importing the dataset, and creating our dependent and independent variables
#we need to split the dataset into the training and test sets, however for this dataset
#we, first, need to encode the categorical variables:

# 2) ENCODING CATEGORICAL DATA: CATEGORICAL TEMPLATE
#For this we have categorical template. This section will endcode any categorical data that you have in your dataset:

# Encoding the Independent Variable: we have two categorical variables
#country and gender

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
#Country column number is 1. This will encode the values as 0, 1, 2...
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
#Gender column number is 2.
#we are creating dummy variables for the country column
#the reason for that we don't want any numbers except 0 or 1. 
#so we will say for example France = 0,0,1; Spain= 1,0,0; German; 0,1,0 instead of  0,1,2
#one hot encoder will create three dummy columns, place it before the credit score column.
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
#we don't need three variables to represent the three different countries, so
#we will delete one dummy variable to avoid falling into the dummy variable trap.
#this won't change the country information: France becomes = 0,0; German=0,1; Spain,1,0
X = X[:, 1:] #this takes all the rows in X (inputs), and takes columns from 1 to end
#CHANGED: just the column numbers, and one delete one dummy variable

# 3) SPLITTING THE DATASET INTO TRAINING AND TEST:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
#CHANGED: just the test size from 0.25 to 0.2 since we have 10,000 observations
#to avoid warning: we changed from sklearn.cross_validation to model_selection

# 4) FEATURE SCALING: 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
#CHANGED: No change

# TUNNING THE ANN

# Part 2: create the ANN with k_fold GridSearch:
##### start time: 5:00pm end time 8:45

In [2]:
#TUNNING THE ANN:
#tunning the hyperparameters
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Using TensorFlow backend.


Epoch 1/100
7200/7200 [==============================] - 1s 100us/step - loss: 0.5633 - acc: 0.7957
Epoch 2/100
7200/7200 [==============================] - 1s 84us/step - loss: 0.4280 - acc: 0.8000
Epoch 3/100
7200/7200 [==============================] - 1s 92us/step - loss: 0.4138 - acc: 0.8207
Epoch 4/100
7200/7200 [==============================] - 1s 85us/step - loss: 0.4041 - acc: 0.8239
Epoch 5/100
7200/7200 [==============================] - 1s 83us/step - loss: 0.3955 - acc: 0.8292
Epoch 6/100
7200/7200 [==============================] - 1s 85us/step - loss: 0.3883 - acc: 0.8311
Epoch 7/100
7200/7200 [==============================] - 1s 82us/step - loss: 0.3827 - acc: 0.8311
Epoch 8/100
7200/7200 [==============================] - 1s 76us/step - loss: 0.3796 - acc: 0.8312
Epoch 9/100
7200/7200 [==============================] - 1s 79us/step - loss: 0.3752 - acc: 0.8335
Epoch 10/100
7200/7200 [==============================] - 1s 86us/step - loss: 0.3731 - acc: 0.8387
Epoch 11

7200/7200 [==============================] - 1s 77us/step - loss: 0.3384 - acc: 0.8624
Epoch 83/100
7200/7200 [==============================] - 1s 72us/step - loss: 0.3386 - acc: 0.8594
Epoch 84/100
7200/7200 [==============================] - 1s 73us/step - loss: 0.3382 - acc: 0.8624
Epoch 85/100
7200/7200 [==============================] - 1s 78us/step - loss: 0.3381 - acc: 0.8611
Epoch 86/100
7200/7200 [==============================] - 0s 69us/step - loss: 0.3379 - acc: 0.8611
Epoch 87/100
7200/7200 [==============================] - 1s 76us/step - loss: 0.3382 - acc: 0.8621
Epoch 88/100
7200/7200 [==============================] - 0s 69us/step - loss: 0.3383 - acc: 0.8594
Epoch 89/100
7200/7200 [==============================] - 0s 69us/step - loss: 0.3371 - acc: 0.8624
Epoch 90/100
7200/7200 [==============================] - 1s 70us/step - loss: 0.3380 - acc: 0.8622
Epoch 91/100
7200/7200 [==============================] - 0s 68us/step - loss: 0.3375 - acc: 0.8615
Epoch 92/100


7200/7200 [==============================] - 1s 100us/step - loss: 0.3928 - acc: 0.8362
Epoch 65/100
7200/7200 [==============================] - 1s 78us/step - loss: 0.3925 - acc: 0.8376
Epoch 66/100
7200/7200 [==============================] - 1s 75us/step - loss: 0.3926 - acc: 0.8371
Epoch 67/100
7200/7200 [==============================] - 1s 78us/step - loss: 0.3923 - acc: 0.8369
Epoch 68/100
7200/7200 [==============================] - 1s 79us/step - loss: 0.3916 - acc: 0.8381
Epoch 69/100
7200/7200 [==============================] - 1s 70us/step - loss: 0.3923 - acc: 0.8361
Epoch 70/100
7200/7200 [==============================] - 1s 72us/step - loss: 0.3924 - acc: 0.8376
Epoch 71/100
7200/7200 [==============================] - 1s 70us/step - loss: 0.3911 - acc: 0.8387
Epoch 72/100
7200/7200 [==============================] - 1s 72us/step - loss: 0.3925 - acc: 0.8360
Epoch 73/100
7200/7200 [==============================] - 1s 79us/step - loss: 0.3920 - acc: 0.8365
Epoch 74/100

7200/7200 [==============================] - 0s 68us/step - loss: 0.4041 - acc: 0.8353
Epoch 47/100
7200/7200 [==============================] - 0s 64us/step - loss: 0.4037 - acc: 0.8357
Epoch 48/100
7200/7200 [==============================] - 0s 65us/step - loss: 0.4037 - acc: 0.8365
Epoch 49/100
7200/7200 [==============================] - 0s 63us/step - loss: 0.4035 - acc: 0.8353
Epoch 50/100
7200/7200 [==============================] - 0s 63us/step - loss: 0.4030 - acc: 0.8367
Epoch 51/100
7200/7200 [==============================] - 0s 64us/step - loss: 0.4036 - acc: 0.8350
Epoch 52/100
7200/7200 [==============================] - 0s 66us/step - loss: 0.4035 - acc: 0.8369
Epoch 53/100
7200/7200 [==============================] - 0s 62us/step - loss: 0.4032 - acc: 0.8358
Epoch 54/100
7200/7200 [==============================] - 0s 63us/step - loss: 0.4031 - acc: 0.8356
Epoch 55/100
7200/7200 [==============================] - 0s 62us/step - loss: 0.4028 - acc: 0.8364
Epoch 56/100


...

8000/8000 [==============================] - 1s 95us/step - loss: 0.4000 - acc: 0.8364
Epoch 98/500
8000/8000 [==============================] - 1s 93us/step - loss: 0.3998 - acc: 0.8355
Epoch 99/500
8000/8000 [==============================] - 1s 93us/step - loss: 0.4000 - acc: 0.8345
Epoch 100/500
8000/8000 [==============================] - 1s 91us/step - loss: 0.3998 - acc: 0.8370
Epoch 101/500
8000/8000 [==============================] - 1s 100us/step - loss: 0.3999 - acc: 0.8349
Epoch 102/500
8000/8000 [==============================] - 1s 93us/step - loss: 0.4000 - acc: 0.8384
Epoch 103/500
8000/8000 [==============================] - 1s 92us/step - loss: 0.4000 - acc: 0.8335
Epoch 104/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3995 - acc: 0.8351
Epoch 105/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3998 - acc: 0.8354
Epoch 106/500
8000/8000 [==============================] - 1s 99us/step - loss: 0.3998 - acc: 0.8361
Epoch

8000/8000 [==============================] - 1s 100us/step - loss: 0.3995 - acc: 0.8357
Epoch 178/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3995 - acc: 0.8366
Epoch 179/500
8000/8000 [==============================] - 1s 92us/step - loss: 0.3997 - acc: 0.8355
Epoch 180/500
8000/8000 [==============================] - 1s 93us/step - loss: 0.3997 - acc: 0.8345
Epoch 181/500
8000/8000 [==============================] - 1s 92us/step - loss: 0.3996 - acc: 0.8370
Epoch 182/500
8000/8000 [==============================] - 1s 101us/step - loss: 0.3995 - acc: 0.8361
Epoch 183/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3992 - acc: 0.8375
Epoch 184/500
8000/8000 [==============================] - 1s 96us/step - loss: 0.3995 - acc: 0.8351
Epoch 185/500
8000/8000 [==============================] - 1s 96us/step - loss: 0.3996 - acc: 0.8364
Epoch 186/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3996 - acc: 0.8365
Ep

8000/8000 [==============================] - 1s 94us/step - loss: 0.3995 - acc: 0.8349
Epoch 259/500
8000/8000 [==============================] - 1s 92us/step - loss: 0.3993 - acc: 0.8355
Epoch 260/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3994 - acc: 0.8356
Epoch 261/500
8000/8000 [==============================] - 1s 102us/step - loss: 0.3993 - acc: 0.8347
Epoch 262/500
8000/8000 [==============================] - 1s 93us/step - loss: 0.3993 - acc: 0.8361
Epoch 263/500
8000/8000 [==============================] - 1s 91us/step - loss: 0.3997 - acc: 0.8370
Epoch 264/500
8000/8000 [==============================] - 1s 93us/step - loss: 0.3987 - acc: 0.8367
Epoch 265/500
8000/8000 [==============================] - 1s 91us/step - loss: 0.3994 - acc: 0.8377
Epoch 266/500
8000/8000 [==============================] - 1s 100us/step - loss: 0.3991 - acc: 0.8377
Epoch 267/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3993 - acc: 0.8374
Ep

8000/8000 [==============================] - 1s 100us/step - loss: 0.3993 - acc: 0.8362
Epoch 340/500
8000/8000 [==============================] - 1s 93us/step - loss: 0.3995 - acc: 0.8360
Epoch 341/500
8000/8000 [==============================] - 1s 93us/step - loss: 0.3988 - acc: 0.8361
Epoch 342/500
8000/8000 [==============================] - 1s 96us/step - loss: 0.3993 - acc: 0.8375
Epoch 343/500
8000/8000 [==============================] - 1s 93us/step - loss: 0.3993 - acc: 0.8350
Epoch 344/500
8000/8000 [==============================] - 1s 92us/step - loss: 0.3990 - acc: 0.8356
Epoch 345/500
8000/8000 [==============================] - 1s 97us/step - loss: 0.3993 - acc: 0.8362
Epoch 346/500
8000/8000 [==============================] - 1s 92us/step - loss: 0.3994 - acc: 0.8354
Epoch 347/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3993 - acc: 0.8359
Epoch 348/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3992 - acc: 0.8357
Epo

8000/8000 [==============================] - 1s 93us/step - loss: 0.3992 - acc: 0.8355
Epoch 421/500
8000/8000 [==============================] - 1s 97us/step - loss: 0.3994 - acc: 0.8356
Epoch 422/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3990 - acc: 0.8370
Epoch 423/500
8000/8000 [==============================] - 1s 94us/step - loss: 0.3993 - acc: 0.8354
Epoch 424/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3991 - acc: 0.8356
Epoch 425/500
8000/8000 [==============================] - 1s 97us/step - loss: 0.3990 - acc: 0.8357
Epoch 426/500
8000/8000 [==============================] - 1s 99us/step - loss: 0.3993 - acc: 0.8356
Epoch 427/500
8000/8000 [==============================] - 1s 95us/step - loss: 0.3993 - acc: 0.8369
Epoch 428/500
8000/8000 [==============================] - 1s 92us/step - loss: 0.3992 - acc: 0.8369
Epoch 429/500
8000/8000 [==============================] - 1s 93us/step - loss: 0.3993 - acc: 0.8366
Epoc

In [3]:
print(best_accuracy)
print(best_parameters)

0.848625
{'batch_size': 25, 'epochs': 500, 'optimizer': 'rmsprop'}
